In [1]:
# from mobile_net import get_model
import librosa
import tensorflow as tf
import glob
import numpy as np
import random
import wave
import os
import pickle
import noisereduce as nr

2024-08-16 06:47:18.481057: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-16 06:47:18.494083: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-16 06:47:18.509557: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-16 06:47:18.514263: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-16 06:47:18.525449: I tensorflow/core/platform/cpu_feature_guar

In [2]:
gpus = tf.config.list_physical_devices('GPU')
gpu = gpus[0]
tf.config.experimental.set_memory_growth(gpu, True)
tf.config.list_logical_devices('GPU')

2024-08-16 06:47:21.818030: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43201 MB memory:  -> device: 0, name: NVIDIA L40S, pci bus id: 0000:8b:00.0, compute capability: 8.9


[LogicalDevice(name='/device:GPU:0', device_type='GPU')]

In [3]:
noise_path = '/shareddrive/working/data_code/data/neg_data/_background_noise_/chunks/1s_chunks/*'
environment_path = '/shareddrive/working/data_code/data/neg_data/envornment/chunks/1s_chunks/*'
word_path = '/shareddrive/working/data_code/data/neg_data/spcmd/all_words/original/*'
recording_path = '/shareddrive/working/data_code/data/neg_data/internet_recordings/chunks/1s_chunks/*'

In [4]:
word_files = glob.glob(word_path)
environment_files = glob.glob(environment_path)
noise_files = glob.glob(noise_path)
recording_files = glob.glob(recording_path)
len(environment_files),len(noise_files),len(word_files),len(recording_files)

(1109, 660, 64721, 5806)

In [5]:
def get_duration(audio_path):
    with wave.open(audio_path, 'rb') as wav_file:
        sample_rate = wav_file.getframerate()
        num_frames = wav_file.getnframes()
        duration = num_frames / sample_rate
    return duration

In [6]:
limit = 1.5
adele_path = '/shareddrive/working/data_code/data/adele/augmented/*'
adele_folders = glob.glob(adele_path)
adele_files = [j for i in adele_folders for j in glob.glob(f'{i}/*')]
# adele_files = [j for i in adele_folders for j in glob.glob(f'{i}/*') if get_duration(j) <= limit]
hilfe_path = '/shareddrive/working/data_code/data/hilfe/augmented/*'
hilfe_folders = glob.glob(hilfe_path)
hilfe_files = [j for i in hilfe_folders for j in glob.glob(f'{i}/*')]
# hilfe_files = [j for i in hilfe_folders for j in glob.glob(f'{i}/*') if get_duration(j) <= limit]

In [7]:
len(adele_files),len(hilfe_files)

(10950, 10935)

In [8]:
no_of_files = 1109+660
file_path_and_labels = list()
# file_path_and_labels.extend([(i,1) for i in random.sample(adele_files,no_of_files)])
file_path_and_labels.extend([(i,1) for i in adele_files])
# file_path_and_labels.extend([(i,2) for i in random.sample(hilfe_files,no_of_files)])
# file_path_and_labels.extend([(i,2) for i in hilfe_files])
# avg_files = (len(adele_files) + len(hilfe_files)) // 2
avg_files = len(adele_files)
# file_path_and_labels.extend([(i,0) for i in random.sample(recording_files,len(recording_files)//2)])
file_path_and_labels.extend([(i,0) for i in random.sample(word_files,avg_files)])
file_path_and_labels.extend([(i,0) for i in noise_files])
file_path_and_labels.extend([(i,0) for i in environment_files])
random.shuffle(file_path_and_labels)
avg_files

10950

In [9]:
avg_files-len(recording_files)//2

8047

In [10]:
sr = 16000
max_seconds = 1
pad_or_trunc = lambda a,i : a[0:i] if len(a) > i else a if len(a) == i else np.pad(a,(0, (i-len(a))))

def process_data(y,sr,max_seconds):
    y = pad_or_trunc(y,sr*max_seconds)
    features = librosa.feature.melspectrogram(y=y,sr=sr,n_fft=1024)
    return features
    
def get_processed_data(audio_file):
    y,_ = librosa.load(audio_file,sr=sr)
    # y = nr.reduce_noise(y=y, sr=sr,n_fft=1024)
    # y[np.isnan(y)] = 0
    features = process_data(y,sr,max_seconds)
    return features
try:
    with open('f_and_l.pickle','rb') as f:
        features_and_labels = pickle.load(f)
except:
    features_and_labels = list()
    for i,j in file_path_and_labels:
        try:
            features = get_processed_data(i)
            features_and_labels.append((features,j))
        except: print(i)
        # features = get_processed_data(i)
        # features_and_labels.append(features,j)
    # with open('f_and_l.pickle','wb') as f:
    #     pickle.dump(features_and_labels,f)
finally:
    print(features_and_labels[0][0].shape)

(128, 32)


In [11]:


class DataSequenceRaw(tf.keras.utils.Sequence):

    def __init__(self,data,batch_size):
        self.data_size = len(data)
        audios,labels = zip(*data)
        self.X = np.array(audios)
        self.Y = np.array(labels)
        self.batch_size = batch_size

        
    def __len__(self):
        return int(np.ceil(self.data_size / self.batch_size))

    def __getitem__(self,idx):
        s = idx * self.batch_size
        e = (idx + 1) * self.batch_size
        X = self.X[s:e]
        Y = self.Y[s:e]
        return X,Y


In [12]:
def _get_train_val_size(total_examples,train_percent=70):
    
    train_ratio = round(train_percent)/100
    remaining_percent = 100 - train_percent
    test_percent = round(remaining_percent/3)*2
    val_percent = remaining_percent - test_percent
    val_ratio = val_percent/100
    train_size = round(total_examples * train_ratio)
    val_size = round(total_examples * val_ratio)
    return train_size,val_size

def get_data_raw(data,train_percent=70,batch_size=32):

    train_size,val_size = _get_train_val_size(len(data),train_percent)

    train_examples = data[:train_size]
    val_examples = data[train_size:train_size+val_size]
    test_examples = data[train_size+val_size:]

    train = DataSequenceRaw(train_examples,batch_size=batch_size)
    test = DataSequenceRaw(test_examples,batch_size=1)
    val = DataSequenceRaw(val_examples,batch_size=batch_size)

    return train,test,val

train,test,val = get_data_raw(features_and_labels,train_percent=80)

In [13]:
shape = train[0][0][0].shape
input_shape = [*shape,1]

In [14]:
def get_model(
        input_shape,
        output_neurons=1,
        output_activation='sigmoid',
        loss=tf.keras.losses.binary_crossentropy,
        lr=0.0001
):
    _input = tf.keras.layers.Input(shape=input_shape)
    x = tf.keras.layers.Conv1D(200,kernel_size=3,padding='valid',activation='relu')(_input)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.GRU(150,return_sequences=False)(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1024,activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(512,activation='relu')(x)
    x = tf.keras.layers.Dense(64,activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(32,activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(10,activation='relu')(x)
    outputs = tf.keras.layers.Dense(output_neurons,activation=output_activation,kernel_regularizer=tf.keras.regularizers.L2(l2=0.01))(x)
    model = tf.keras.Model(inputs=_input,outputs=outputs)

    model.compile(
        loss=loss,
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        metrics=['accuracy'],
    )

    return model

model = get_model(
        input_shape=shape,
        lr=0.001
)



In [15]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy',factor=0.1,patience=5,mode='max')
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=1,mode='max',restore_best_weights=True,start_from_epoch=10)
with tf.device('/gpu'):
    history = model.fit(train,epochs=100,validation_data=val,verbose=1,callbacks=[reduce_lr,early_stopping])

Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-08-16 06:56:40.120580: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
W0000 00:00:1723791400.229350     557 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723791400.269781     557 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723791400.270037     557 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723791400.270255     557 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723791400.282679  

 24/592 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.4973 - loss: 0.7134

W0000 00:00:1723791401.833619     557 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723791401.833947     557 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723791401.834420     557 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723791401.834782     557 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723791401.835586     557 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723791401.836423     557 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723791401.836639     557 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723791401.837618     557 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723791401.837833     557 gp

390/592 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5517 - loss: 0.7013

W0000 00:00:1723791404.363851     557 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723791404.364095     557 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723791404.364291     557 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723791404.364476     557 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723791404.364683     557 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723791404.364855     557 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723791404.365020     557 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723791404.365198     557 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723791404.365374     557 gp

592/592 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.5761 - loss: 0.6871 - val_accuracy: 0.7282 - val_loss: 0.5435 - learning_rate: 0.0010
Epoch 2/100
  1/592 ━━━━━━━━━━━━━━━━━━━━ 16s 27ms/step - accuracy: 0.7500 - loss: 0.6268

W0000 00:00:1723791406.102001     556 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723791406.107441     556 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723791406.107640     556 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723791406.107827     556 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723791406.108021     556 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723791406.108280     556 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723791406.108471     556 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723791406.108657     556 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723791406.108851     556 gp

592/592 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.8968 - loss: 0.3127 - val_accuracy: 0.9704 - val_loss: 0.0977 - learning_rate: 0.0010
Epoch 3/100
592/592 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.9639 - loss: 0.1383 - val_accuracy: 0.9782 - val_loss: 0.0873 - learning_rate: 0.0010
Epoch 4/100
592/592 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.9690 - loss: 0.1122 - val_accuracy: 0.9725 - val_loss: 0.0846 - learning_rate: 0.0010
Epoch 5/100
592/592 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.9727 - loss: 0.1023 - val_accuracy: 0.9662 - val_loss: 0.1112 - learning_rate: 0.0010
Epoch 6/100
592/592 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.9781 - loss: 0.0846 - val_accuracy: 0.9718 - val_loss: 0.0711 - learning_rate: 0.0010
Epoch 7/100
592/592 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.9798 - loss: 0.0675 - val_accuracy: 0.9852 - val_loss: 0.0529 - learning_rate: 0.0010
Epoch 8/100
592/592 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.9810 - loss: 0.0627 - val_a

In [18]:
model.evaluate(test)

3314/3314 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9868 - loss: 0.0431


[0.046048372983932495, 0.9864212274551392]

In [19]:
folder_path = '/shareddrive/working/model_code/models/custom_model_3/trail_1'
os.makedirs(folder_path,exist_ok=True)
model_path = f'{folder_path}/16k_melspec-nfft-1024_a_cnn_gru_dense_model.keras'
model.save(model_path)